# Cell phone MIP problem in parametric form

This notebook shows how to model and solve the cell phone factory production MIP using a more parametric approach, i.e., a general one where data could be read from a file.

## Mobile phone factory

The XYZ mobile corporation produces two models of cell phone, which we'll call M1 and M2, from a pool of components. Both M1 and M2 have the basic components (display, memory, camera, CPU), while only one model, M1, has two extras: thermal FLIR camera and satellite rx/tx.

The only factory of XYZ has a limited supply of each component, and the number of components for each model is described in the following table

Component|M1|M2|Availability
---------|--|--|------------
Display|1|2|10
Memory|2|2|18
Camera|1|3|12
CPU|2|3|21
Thermal cam.|1|0|9
Satellite rx/tx|1|0|10

The sales price for M1 is 110, and for M2 it is 130. Formulate the problem of finding how many models to produce of M1 and M2 in order to mazimize the total revenue.


This problem can be modeled in a simple way. First, the main decision consists in two quantities: the number of M1 and the number of M2 to produce. We assign two variables $x_1$ and $x_2$ to these quantities.

Next, the optimization model will have $110 x_1 + 130 x_2$ as objective function, which should be maximized. Finally, the constraints are given by each scarse resource (displays, memories, etc.). One constraint can be given for each resource. For instance, given that there are 10 displays in total and M1 uses one while M2 uses two, this implies the constraint

$$
x_1 + 2x_2 \le 10
$$

And similarly for all other resources. The two variables $x_1$ and $x_2$ must obviously be nonnegative and integer. The final model can be written as follows:

$$
\begin{array}{llll}
\max & 110 x_1 + 130 x_2\\
\textrm{s.t.} &   x_1 + 2 x_2 & \le 10&\qquad\textrm{(display)}\\
              & 2 x_1 + 2 x_2 & \le 18&\qquad\textrm{(memory)}\\
              &   x_1 + 3 x_2 & \le 12&\qquad\textrm{(camera)}\\
              & 2 x_1 + 3 x_2 & \le 21&\qquad\textrm{(CPU)}\\
              &   x_1         & \le 9 &\qquad\textrm{(thermal camera)}\\
              &   x_2         & \le 10&\qquad\textrm{(sat. rx/tx)}\\
              & x_1, x_2 \in \mathbb Z_+.
\end{array}
$$

In matricial form, one can define

$$
x = \left(\begin{array}{l}x_1\\x_2\end{array}\right);\qquad
c = \left(\begin{array}{r}110\\130\end{array}\right);\qquad
A = \left(\begin{array}{rr}1&2\\2&2\\1&3\\2&3\\1&0\\1&0\\\end{array}\right);\qquad
b = \left(\begin{array}{r}10\\18\\12\\21\\9\\10\\\end{array}\right),
$$

and the model can be re-written equivalently as

$$
\begin{array}{lll}
\max          & c^T x\\
\textrm{s.t.} & A x \le b\\
              & x \in \mathbb Z_+^2
\end{array}
$$

We are ready to create an optimization problem using the `mip` module. We write a more general version where the data is specified separately, so that it could even be read from a file. Note that the component usage can be created as a 6x2 "matrix" simply by creating a list of six elements, each being a list of two. 

In [1]:
# When using Colab, make sure you run this instruction beforehand
!pip install --upgrade cffi==1.15.0
import importlib
import cffi
importlib.reload(cffi)
!pip install mip

In [16]:
import mip

n_model = 2 # Number of variables. 
n_component = 6 # Number of constraints.

c = [110, 130]

A = [[1,2],
     [2,2],
     [1,3],
     [2,3],
     [1,0],
     [1,0]]# Complete (TODO)

b = [10,18,12,21,9,10]

2


Now we create an empty model and add the two integer variables $[x_1,x_2]$.

In [25]:
m = mip.Model()
# Instantiate x variables (TODO)
x = [m.add_var(var_type=mip.INTEGER) for i in range(n_model)]


Let's write the objective function: in the general case, it can be written as a sum over the set of models.

In [26]:
m.objective = mip.maximize(110*x[0] + 130*x[1])

The constraints can be generated in a loop:

In [27]:
for j in range(n_component):
    m.add_constr(mip.xsum([A[j][i]*x[i] for i in range(n_model)]) <= b[j])  # Complete in the "..." (TODO)

The model is complete. Let's solve it and print the optimal solution.

In [28]:
status = m.optimize()

# Print the value (.x) of each variable
print([x[i].x for i in range(n_model)])
# Print the objective function value
print(f"The price is : {m.objective_value} €)
print(status)

Clp0002I Dual infeasible - objective value -0
Clp0032I DualInfeasible objective -0 - 0 iterations time 0.002, Idiot 0.00
Starting solution of the Linear programming relaxation problem using Primal Simplex

Clp0024I Matrix will be packed to eliminate 2 small elements
Coin0506I Presolve 4 (-2) rows, 2 (0) columns and 8 (-2) elements
Clp1000I sum of infeasibilities 0 - average 0, 0 fixed columns
Coin0506I Presolve 4 (0) rows, 2 (0) columns and 8 (0) elements
Clp0006I 0  Obj 1010 Dual inf 24000 (2)
Clp0029I End of values pass after 2 iterations
Clp0000I Optimal - objective value 1010
Clp0000I Optimal - objective value 1010
Clp0000I Optimal - objective value 1010
Coin0511I After Postsolve, objective 1010, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 1010 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
[8.0, 1.0]
1010.0
OptimizationStatus.OPTIMAL
